In [56]:
#First part
import os
import json
import codecs
#Get all the files in the directory
data_path1 = 'data/2015-03-18'
data_path2 = 'data/2017-12-20'

files1 = os.listdir(data_path1)
files2 = os.listdir(data_path2)

names = []
titles = []
prices = []
currencies = []
total_value = {}

#Parse through the files
filepaths = []

for f in files1:
    file_path = data_path1 + '/' + f
    filepaths.append(file_path)
    
for f in files2:
    file_path = data_path2 + '/' + f
    filepaths.append(file_path)
    
for file_path in filepaths:
    with codecs.open(file_path,'r', encoding = 'utf-8') as file:
        prev_line = ''
        for line in file:
            if '<title>' in prev_line: #getting the title and artist
                titleline = line.lstrip()
                for i in range(len(titleline)):
                    c = titleline[i]
                    if c == ':': #colon seperates title and artist
                        break
                name = titleline[:i]
                title = titleline[i+2:-1]
                
                #Some of the names have dates in them, so I just remove them
                for i in range(len(name)):
                    c = name[i]
                    if c == '(':
                        name = name[:i-1]
                        break
                        
            if 'Price realised' in prev_line: # getting the price
                priceline = line.lstrip()
                price = ''.join(c for c in priceline if c.isdigit())
                currency = priceline[5:8]
            prev_line = line
        
        #Store the values for this html file
        
        #convert currency
        if currency != 'USD':
            currency = 'USD'
            price = str(round(int(price) / 1.34))
        
        names.append(name)
        titles.append(title)
        prices.append(price)
        currencies.append(currency)
        
        #get total value
        if name in total_value.keys():
            total_value[name] += int(price)
        else:
            total_value[name] = int(price)

#Convert currency
works = [{'title': title, 'currency': currency, 'amount': price} for title, currency, price in zip(titles, currencies, prices)]
        
#Combine the author's works to be shown together
nameset = list(set(names))

artist_works = []
artist_totals = []
for i in range(len(nameset)):
    artist_works.append([])
    name = names[i]
    artist_totals.append('USD ' + str(total_value[name]))

for i in range(len(names)):
    name = names[i]
    name_index = nameset.index(name)
    work = works[i]
    artist_works[name_index].append(work)
 
output = [{'artist': artist, 'totalValue': value, 'works': work} for artist, value, work in zip(nameset, artist_totals, artist_works)]

output = json.dumps(output)
print(output)


[{"artist": "Louis Marcoussis", "totalValue": "USD 9609", "works": [{"title": "La Table", "currency": "USD", "amount": "6866"}, {"title": "Portrait d'Alice Halicka", "currency": "USD", "amount": "4478"}, {"title": "Bar du porto", "currency": "USD", "amount": "4478"}]}, {"artist": "Rembrandt Harmensz. van Rijn", "totalValue": "USD 15822", "works": [{"title": "Christ at Emmaus: The smaller Plate", "currency": "USD", "amount": "5131"}, {"title": "Self-Portrait in a Cap and Scarf with Face dark: Bust", "currency": "USD", "amount": "4478"}]}, {"artist": "Marc Chagall", "totalValue": "USD 8396", "works": [{"title": "Self-Portrait", "currency": "USD", "amount": "4478"}]}, {"artist": "Joan Mir\u00f3", "totalValue": "USD 33956", "works": [{"title": "Femme et Chien devant la Lune", "currency": "USD", "amount": "8396"}]}, {"artist": "Pablo Picasso", "totalValue": "USD 4478", "works": [{"title": "Quatre Femmes nues et T\u00eate sculpt\u00e9e, from: La Suite Vollard", "currency": "USD", "amount": "